In [4]:
import pandas as pd
import polars as pl
import pyarrow as pa
from datetime import datetime, timedelta
import random
import numpy as np
import os
import db_dtypes
import pprint as pp

In [5]:
try:
    from google.cloud import bigquery
    print("google.cloud.bigquery imported successfully.")
except ImportError:
    print("google-cloud-bigquery not found. Please install it: pip install google-cloud-bigquery")
    bigquery = None # Set to None if import fails

google.cloud.bigquery imported successfully.


In [6]:
def fetch_bigquery_data(project_id, dataset_id, table_id, columns=None, limit=None):
    """
    Fetches data from a BigQuery table and returns it as a Polars DataFrame.

    Args:
        project_id (str): Your Google Cloud Project ID.
        dataset_id (str): The ID of the BigQuery dataset.
        table_id (str): The ID of the BigQuery table.
        columns (list of str, optional): A list of column names to select. If None, all columns are selected.
        limit (int, optional): The maximum number of rows to return. If None, no limit is applied.

    Returns:
        polars.DataFrame or None: A Polars DataFrame containing the fetched data, or None if an error occurs.
    """
    # Ensure bigquery client is available (check if the library was imported successfully)
    # This check assumes 'bigquery' is imported globally or passed in.
    # A more robust check might involve checking for google.cloud.bigquery.Client
    try:
        client = bigquery.Client(project=project_id)
    except Exception as e:
        print(f"Error initializing BigQuery client: {e}")
        print("Please ensure the 'google-cloud-bigquery' library is installed and configured.")
        return None

    try:
        # Construct the fully qualified table ID for the query
        # No need to explicitly construct table_ref for the query string itself,
        # but client initialization handles project context.

        # Build the SELECT statement
        if columns:
            select_cols = ", ".join(columns)
        else:
            select_cols = "*"

        query = f"SELECT {select_cols} FROM `{project_id}.{dataset_id}.{table_id}`"
        if limit:
            query += f" LIMIT {limit}"

        print(f"Executing BigQuery query: {query}")
        query_job = client.query(query)

        # Wait for the job to complete and fetch results as an Arrow Table
        # The .to_arrow() method is efficient and ideal for Polars conversion.
        # This implicitly uses the BigQuery Storage API if installed and available.
        arrow_table = query_job.result().to_arrow()

        # Convert the PyArrow Table to a Polars DataFrame
        df = pl.DataFrame(arrow_table)
        return df

    except Exception as e:
        print(f"Error fetching data from BigQuery: {e}")
        print("Ensure your GCP project, dataset, table IDs are correct, and authentication is configured (e.g., GOOGLE_APPLICATION_CREDENTIALS).")
        return None



In [7]:
# --- Configuration for dummy data generation (fallback if BigQuery fails) ---
NUM_ORDERS = 125000
# NUM_ORDERS = 10
NUM_CUSTOMERS = 100000
NUM_PRODUCTS = 35000
NUM_SELLERS = 3500


# --- Attempt to fetch real data, fallback to dummy if needed ---
orders_df_pl = None
order_items_df_pl = None
orders_with_items_df_pl = None

order_reviews_df_pl = None
products_df_pl = None
payments_df_pl = None
customers_df_pl = None
orders_loyalty_df_pl = None # Specific for loyalty insight

In [8]:
print("\n--- Attempting to fetch real data from BigQuery ---")

# Replace these with your actual BigQuery details
BIGQUERY_PROJECT_ID = "meltano-learn-461903" # e.g., "my-data-project-12345"
BIGQUERY_DATASET_ID = "olist_raw"   # e.g., "ecommerce_data"


--- Attempting to fetch real data from BigQuery ---


In [29]:
# Fetch orders data

orders_df_pl = fetch_bigquery_data(
    project_id=BIGQUERY_PROJECT_ID,
    dataset_id=BIGQUERY_DATASET_ID,
    table_id="olist_orders", # e.g., "orders"
    # columns=['order_id', 'customer_id', 'order_purchase_timestamp', 'price', 'freight_value'],
    columns=['order_id', 'customer_id', 'order_purchase_timestamp','order_estimated_delivery_date', 'order_delivered_customer_date'],
    limit=NUM_ORDERS
)

date_format = "%Y-%m-%d %H:%M:%S"

if orders_df_pl.schema['order_purchase_timestamp'] == pl.Utf8:
    orders_df_pl = orders_df_pl.with_columns(
        pl.col('order_purchase_timestamp')
        .str.strptime(pl.Datetime, date_format, strict=False) # strict=False is good for parsing flexibility
        .alias('order_purchase_timestamp')
    )

if orders_df_pl.schema['order_estimated_delivery_date'] == pl.Utf8:
    orders_df_pl = orders_df_pl.with_columns(
        pl.col('order_estimated_delivery_date')
        .str.strptime(pl.Datetime, date_format, strict=False) # strict=False is good for parsing flexibility
        .alias('order_estimated_delivery_date')
    )

if orders_df_pl.schema['order_delivered_customer_date'] == pl.Utf8:
    orders_df_pl = orders_df_pl.with_columns(
        pl.col('order_delivered_customer_date')
        .str.strptime(pl.Datetime, date_format, strict=False) # strict=False is good for parsing flexibility
        .alias('order_delivered_customer_date')
    )

pp.pprint(orders_df_pl.head(5))

Executing BigQuery query: SELECT order_id, customer_id, order_purchase_timestamp, order_estimated_delivery_date, order_delivered_customer_date FROM `meltano-learn-461903.olist_raw.olist_orders` LIMIT 125000
shape: (5, 5)
┌───────────────────┬───────────────────┬───────────────────┬───────────────────┬──────────────────┐
│ order_id          ┆ customer_id       ┆ order_purchase_ti ┆ order_estimated_d ┆ order_delivered_ │
│ ---               ┆ ---               ┆ mestamp           ┆ elivery_date      ┆ customer_date    │
│ str               ┆ str               ┆ ---               ┆ ---               ┆ ---              │
│                   ┆                   ┆ datetime[μs]      ┆ datetime[μs]      ┆ datetime[μs]     │
╞═══════════════════╪═══════════════════╪═══════════════════╪═══════════════════╪══════════════════╡
│ a2e4c44360b4a57bd ┆ 8886130db0ea6e9e7 ┆ 2017-02-06        ┆ 2017-03-01        ┆ null             │
│ ff22f3a4630c173   ┆ 0ba0b03d7c0d286   ┆ 20:18:17          ┆ 00:00:00  

In [30]:
order_items_df_pl = fetch_bigquery_data(
    project_id=BIGQUERY_PROJECT_ID,
    dataset_id=BIGQUERY_DATASET_ID,
    table_id="olist_order_items", # e.g., "order_items"
    columns=['order_id', 'product_id', 'seller_id', 'price','freight_value'],
    limit=NUM_ORDERS
)

# Explicitly cast 'price' and 'freight_value' as Float64
order_items_df_pl = order_items_df_pl.with_columns([
    pl.col("price").cast(pl.Float64),
    pl.col("freight_value").cast(pl.Float64)
])

pp.pprint(order_items_df_pl.head(5))

Executing BigQuery query: SELECT order_id, product_id, seller_id, price, freight_value FROM `meltano-learn-461903.olist_raw.olist_order_items` LIMIT 125000
shape: (5, 5)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────┬───────────────┐
│ order_id               ┆ product_id             ┆ seller_id              ┆ price ┆ freight_value │
│ ---                    ┆ ---                    ┆ ---                    ┆ ---   ┆ ---           │
│ str                    ┆ str                    ┆ str                    ┆ f64   ┆ f64           │
╞════════════════════════╪════════════════════════╪════════════════════════╪═══════╪═══════════════╡
│ 3ee6513ae7ea23bdfab5b9 ┆ 8a3254bee785a526d548a8 ┆ 96804ea39d96eb908e7c3a ┆ 0.85  ┆ 18.23         │
│ ab60bffcb5             ┆ 1a9bc3c9be             ┆ fdb671bb9e             ┆       ┆               │
│ c5bdd8ef3c0ec420232e66 ┆ 8a3254bee785a526d548a8 ┆ 96804ea39d96eb908e7c3a ┆ 0.85  ┆ 22.3          │
│ 8302179113          

In [31]:
orders_with_items_df_pl = orders_df_pl.join(
    order_items_df_pl,
    on="order_id",
    how="left" # Use 'left' to keep all orders from orders_df_pl
)

pp.pprint(orders_with_items_df_pl.head(5))

shape: (5, 9)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────┬───────────┐
│ order_id   ┆ customer_i ┆ order_purc ┆ order_est ┆ … ┆ product_i ┆ seller_id ┆ price ┆ freight_v │
│ ---        ┆ d          ┆ hase_times ┆ imated_de ┆   ┆ d         ┆ ---       ┆ ---   ┆ alue      │
│ str        ┆ ---        ┆ tamp       ┆ livery_da ┆   ┆ ---       ┆ str       ┆ f64   ┆ ---       │
│            ┆ str        ┆ ---        ┆ te        ┆   ┆ str       ┆           ┆       ┆ f64       │
│            ┆            ┆ datetime[μ ┆ ---       ┆   ┆           ┆           ┆       ┆           │
│            ┆            ┆ s]         ┆ datetime[ ┆   ┆           ┆           ┆       ┆           │
│            ┆            ┆            ┆ μs]       ┆   ┆           ┆           ┆       ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════╪═══════════╡
│ a2e4c44360 ┆ 8886130db0 ┆ 2017-02-06 ┆ 2017-03-0 ┆ … ┆ dcecb611b ┆ 8cc6a0e5

In [12]:
# Fetch other tables similarly
order_reviews_df_pl = fetch_bigquery_data(
    project_id=BIGQUERY_PROJECT_ID,
    dataset_id=BIGQUERY_DATASET_ID,
    table_id="olist_order_reviews", # e.g., "order_reviews"
    columns=['review_id', 'order_id', 'review_score'],
    limit=NUM_ORDERS 
)

# Explicitly cast 'review_score' as Int64
order_reviews_df_pl = order_reviews_df_pl.with_columns([
    pl.col("review_score").cast(pl.Int64)
])

pp.pprint(order_reviews_df_pl.head(5))

Executing BigQuery query: SELECT review_id, order_id, review_score FROM `meltano-learn-461903.olist_raw.olist_order_reviews` LIMIT 125000
shape: (5, 3)
┌──────────────────────────────────┬──────────────────────────────────┬──────────────┐
│ review_id                        ┆ order_id                         ┆ review_score │
│ ---                              ┆ ---                              ┆ ---          │
│ str                              ┆ str                              ┆ i64          │
╞══════════════════════════════════╪══════════════════════════════════╪══════════════╡
│ d7486b4da781b9d7dc5484ce65868081 ┆ 45973912e490866800c0aea8f63099c8 ┆ 1            │
│ 65f4e79529365e241d4514d436ed50ba ┆ 73c182f8914a0064d6c64ad65e4c6942 ┆ 1            │
│ 40dad6438b6cbec46d936bec2377778c ┆ bb5849f8ba21da43ffa31ea52ba81b37 ┆ 1            │
│ 3f302efaba2d6b4cc696158658834c46 ┆ 2df094c40051a8f5ed95344421039ced ┆ 1            │
│ 531c9a1377149f5e3e29c5c6d6252614 ┆ 14d45fea1c20e4fd5c31dde98334

In [13]:
products_df_pl = fetch_bigquery_data(
    project_id=BIGQUERY_PROJECT_ID,
    dataset_id=BIGQUERY_DATASET_ID,
    table_id="olist_products", # e.g., "products"
    columns=['product_id', 'product_category_name'],
    limit=NUM_PRODUCTS
)

pp.pprint(products_df_pl.head(5))

Executing BigQuery query: SELECT product_id, product_category_name FROM `meltano-learn-461903.olist_raw.olist_products` LIMIT 35000
shape: (5, 2)
┌──────────────────────────────────┬───────────────────────┐
│ product_id                       ┆ product_category_name │
│ ---                              ┆ ---                   │
│ str                              ┆ str                   │
╞══════════════════════════════════╪═══════════════════════╡
│ 7d0e0214a0852cf0c90032c7992cf27f ┆ market_place          │
│ e4f61b606514ddd78124fd38f3433155 ┆ telefonia_fixa        │
│ b0694770dbbfebe2d48cd288cfce2f85 ┆ artes                 │
│ 4eda4a7eac36a8ff69db8880c4d95b7b ┆ livros_tecnicos       │
│ 4e5696c21b0c174738e7929b147b7126 ┆ moveis_sala           │
└──────────────────────────────────┴───────────────────────┘


In [14]:
payments_df_pl = fetch_bigquery_data(
    project_id=BIGQUERY_PROJECT_ID,
    dataset_id=BIGQUERY_DATASET_ID,
    table_id="olist_order_payments", # e.g., "order_payments"
    columns=['order_id', 'payment_type', 'payment_value'],
    limit=NUM_ORDERS
)

# Explicitly cast 'payment_value' as Float64
payments_df_pl = payments_df_pl.with_columns([
    pl.col("payment_value").cast(pl.Float64)
])

pp.pprint(payments_df_pl.head(5))

Executing BigQuery query: SELECT order_id, payment_type, payment_value FROM `meltano-learn-461903.olist_raw.olist_order_payments` LIMIT 125000
shape: (5, 3)
┌──────────────────────────────────┬──────────────┬───────────────┐
│ order_id                         ┆ payment_type ┆ payment_value │
│ ---                              ┆ ---          ┆ ---           │
│ str                              ┆ str          ┆ f64           │
╞══════════════════════════════════╪══════════════╪═══════════════╡
│ 25e8ea4e93396b6fa0d3dd708e76c1bd ┆ credit_card  ┆ 65.71         │
│ b81ef226f3fe1789b1e8b2acac839d17 ┆ credit_card  ┆ 99.33         │
│ a9810da82917af2d9aefd1278f1dcfa0 ┆ credit_card  ┆ 24.39         │
│ ba78997921bbcdc1373bb41e913ab953 ┆ credit_card  ┆ 107.78        │
│ 0573b5e23cbd798006520e1d5b4c6714 ┆ boleto       ┆ 51.95         │
└──────────────────────────────────┴──────────────┴───────────────┘


In [15]:
customers_df_pl = fetch_bigquery_data(
    project_id=BIGQUERY_PROJECT_ID,
    dataset_id=BIGQUERY_DATASET_ID,
    table_id="olist_customers", # e.g., "customers"
    columns=['customer_id', 'customer_unique_id', 'customer_state'],
    limit=NUM_CUSTOMERS
)

pp.pprint(customers_df_pl.head(5))

Executing BigQuery query: SELECT customer_id, customer_unique_id, customer_state FROM `meltano-learn-461903.olist_raw.olist_customers` LIMIT 100000
shape: (5, 3)
┌──────────────────────────────────┬──────────────────────────────────┬────────────────┐
│ customer_id                      ┆ customer_unique_id               ┆ customer_state │
│ ---                              ┆ ---                              ┆ ---            │
│ str                              ┆ str                              ┆ str            │
╞══════════════════════════════════╪══════════════════════════════════╪════════════════╡
│ 8a0108267d9258a0ec9f74381bc9b0de ┆ 7a2dc4682890550ebe3b8befcea3d55c ┆ AC             │
│ 888d2ebe1af2a8c93c75dae5dfc23719 ┆ 721d1092e1a6460c67e6a0e691d899a3 ┆ AC             │
│ 2d618e470c95c9b425cbb0cbc42683dd ┆ f7d7fc0a59ef4363fdce6e3aa069d498 ┆ AC             │
│ 0f32385df13e46d88d997460208bc866 ┆ 4f67110f6d6d1241111167b141bfa780 ┆ AC             │
│ 53996870173a3a001a1fb56ef00b9150 ┆ 

In [23]:
orders_loyalty_df_pl = orders_df_pl.join(
    customers_df_pl,
    on="customer_id",
    how="left" # Use a left join to keep all orders from orders_df_pl
)

In [16]:
# --- Start of EDA Scripts (using the now available DataFrames) ---

# --- 1. Total Revenue, Total Orders, Average Order Value ---
print("--- Insight 1: Total Revenue, Total Orders, Average Order Value ---")
# print("\nPandas Implementation:")
# total_revenue_pl = (order_items_df_pl['price'] + order_items_df_pl['freight_value']).sum()
# total_orders_pl = orders_df_pl['order_id'].unique()
# average_order_value_pl = total_revenue_pl / total_orders_pl
# print(f"Total Revenue: R${total_revenue_pl:,.2f}")
# print(f"Total Orders: {total_orders_pl}")
# print(f"Average Order Value: R${average_order_value_pl:,.2f}")

print("\nPolars Implementation:")
total_revenue_pl = order_items_df_pl.with_columns(
    (pl.col('price') + pl.col('freight_value')).alias('total_price')
).select(pl.sum('total_price')).item()
total_orders_pl = orders_df_pl.select(pl.n_unique('order_id')).item()

average_order_value_pl = total_revenue_pl / total_orders_pl
print(f"Total Revenue: R${total_revenue_pl:,.2f}")
print(f"Total Orders: {total_orders_pl}")
print(f"Average Order Value: R${average_order_value_pl:,.2f}")

--- Insight 1: Total Revenue, Total Orders, Average Order Value ---

Polars Implementation:
Total Revenue: R$15,843,553.24
Total Orders: 99441
Average Order Value: R$159.33


In [17]:
# --- 2. Average Product Rating ---
print("\n--- Insight 2: Average Product Rating ---")
# print("\nPandas Implementation:")
# average_rating_pd = order_reviews_df_pd['review_score'].mean()
# print(f"Average Product Rating: {average_rating_pd:.2f} ★")

print("\nPolars Implementation:")
average_rating_pl = order_reviews_df_pl.select(pl.mean('review_score')).item()
print(f"Average Product Rating: {average_rating_pl:.2f} ★")


--- Insight 2: Average Product Rating ---

Polars Implementation:
Average Product Rating: 4.09 ★


In [33]:
# --- 3. Sales Growth Trajectory (Monthly Revenue Trend) ---
print("\n--- Insight 3: Sales Growth Trajectory (Monthly Revenue Trend) ---")
# print("\nPandas Implementation:")
# orders_df_pd['order_purchase_timestamp'] = pd.to_datetime(orders_df_pd['order_purchase_timestamp'])
# orders_df_pd['month_year'] = orders_df_pd['order_purchase_timestamp'].dt.to_period('M')
# orders_df_pd['total_price'] = orders_df_pd['price'] + orders_df_pd['freight_value']
# monthly_revenue_pd = orders_df_pd.groupby('month_year')['total_price'].sum().reset_index()
# monthly_revenue_pd['month_year'] = monthly_revenue_pd['month_year'].astype(str)
# print("Monthly Revenue Trend (Pandas):\n", monthly_revenue_pd.tail())

print("\nPolars Implementation:")

monthly_revenue_pl = orders_with_items_df_pl.with_columns(
    pl.col('order_purchase_timestamp').dt.truncate('1mo').alias('month_year'),
    (pl.col('price') + pl.col('freight_value')).alias('total_price')
).group_by('month_year').agg(pl.sum('total_price')).sort('month_year')
print("Monthly Revenue Trend (Polars):\n", monthly_revenue_pl.tail())


--- Insight 3: Sales Growth Trajectory (Monthly Revenue Trend) ---

Polars Implementation:
Monthly Revenue Trend (Polars):
 shape: (5, 2)
┌─────────────────────┬─────────────┐
│ month_year          ┆ total_price │
│ ---                 ┆ ---         │
│ datetime[μs]        ┆ f64         │
╞═════════════════════╪═════════════╡
│ 2018-06-01 00:00:00 ┆ 1.0227e6    │
│ 2018-07-01 00:00:00 ┆ 1.0587e6    │
│ 2018-08-01 00:00:00 ┆ 1.0033e6    │
│ 2018-09-01 00:00:00 ┆ 166.46      │
│ 2018-10-01 00:00:00 ┆ 0.0         │
└─────────────────────┴─────────────┘


In [19]:
# --- 4. Top Selling Product Categories (Number of Orders) ---
print("\n--- Insight 4: Top Selling Product Categories ---")
# print("\nPandas Implementation:")
# merged_df_pd = pd.merge(order_items_df_pd, products_df_pd, on='product_id', how='left')
# top_categories_pd = merged_df_pd.groupby('product_category_name')['order_id'].nunique().sort_values(ascending=False).head(5)
# print("Top Product Categories by Orders (Pandas):\n", top_categories_pd)

print("\nPolars Implementation:")
merged_df_pl = order_items_df_pl.join(products_df_pl, on='product_id', how='left')
top_categories_pl = merged_df_pl.group_by('product_category_name').agg(
    pl.n_unique('order_id').alias('unique_orders')
).sort('unique_orders', descending=True).head(5)
print("Top Product Categories by Orders (Polars):\n", top_categories_pl)


--- Insight 4: Top Selling Product Categories ---

Polars Implementation:
Top Product Categories by Orders (Polars):
 shape: (5, 2)
┌────────────────────────┬───────────────┐
│ product_category_name  ┆ unique_orders │
│ ---                    ┆ ---           │
│ str                    ┆ u32           │
╞════════════════════════╪═══════════════╡
│ cama_mesa_banho        ┆ 9417          │
│ beleza_saude           ┆ 8836          │
│ esporte_lazer          ┆ 7720          │
│ informatica_acessorios ┆ 6689          │
│ moveis_decoracao       ┆ 6449          │
└────────────────────────┴───────────────┘


In [20]:
# --- 5. Payment Method Usage (Percentage) ---
print("\n--- Insight 5: Payment Method Usage ---")
# print("\nPandas Implementation:")
# payment_counts_pd = payments_df_pd['payment_type'].value_counts()
# payment_percentages_pd = payment_counts_pd / payment_counts_pd.sum() * 100
# print("Payment Method Usage (Pandas):\n", payment_percentages_pd)

print("\nPolars Implementation:")

# 1. Group by 'payment_type' and aggregate by counting the length of each group
#    Use pl.count() as an aggregation expression within .agg()
payment_counts_pl = payments_df_pl.group_by('payment_type').agg(
    pl.count().alias('count') # This counts all rows in the group
)

# 2. Calculate the total number of payments (sum of the 'count' column)
total_payments_pl = payment_counts_pl.select(pl.sum('count')).item()

# 3. Calculate percentages
payment_percentages_pl = payment_counts_pl.with_columns(
    (pl.col('count') / total_payments_pl * 100).alias('percentage')
)

print("Payment Method Usage (Polars):\n", payment_percentages_pl)


--- Insight 5: Payment Method Usage ---

Polars Implementation:
Payment Method Usage (Polars):
 shape: (5, 3)
┌──────────────┬───────┬────────────┐
│ payment_type ┆ count ┆ percentage │
│ ---          ┆ ---   ┆ ---        │
│ str          ┆ u32   ┆ f64        │
╞══════════════╪═══════╪════════════╡
│ not_defined  ┆ 3     ┆ 0.002888   │
│ credit_card  ┆ 76795 ┆ 73.922376  │
│ debit_card   ┆ 1529  ┆ 1.471806   │
│ voucher      ┆ 5775  ┆ 5.558978   │
│ boleto       ┆ 19784 ┆ 19.043952  │
└──────────────┴───────┴────────────┘


In [21]:
# --- 6. Geographical Sales Hotspot (São Paulo) ---
print("\n--- Insight 6: Geographical Sales Hotspot ---")
# print("\nPandas Implementation:")
# orders_customers_pd = pd.merge(orders_df_pd, customers_df_pd, on='customer_id', how='left')
# sales_by_state_pd = orders_customers_pd.groupby('customer_state')['order_id'].nunique().sort_values(ascending=False).head(5)
# print("Top States by Orders (Pandas):\n", sales_by_state_pd)

print("\nPolars Implementation:")
orders_customers_pl = orders_df_pl.join(customers_df_pl, on='customer_id', how='left')
sales_by_state_pl = orders_customers_pl.group_by('customer_state').agg(
    pl.n_unique('order_id').alias('unique_orders')
).sort('unique_orders', descending=True).head(5)
print("Top States by Orders (Polars):\n", sales_by_state_pl)


--- Insight 6: Geographical Sales Hotspot ---

Polars Implementation:
Top States by Orders (Polars):
 shape: (5, 2)
┌────────────────┬───────────────┐
│ customer_state ┆ unique_orders │
│ ---            ┆ ---           │
│ str            ┆ u32           │
╞════════════════╪═══════════════╡
│ SP             ┆ 41746         │
│ RJ             ┆ 12852         │
│ MG             ┆ 11635         │
│ RS             ┆ 5466          │
│ PR             ┆ 5045          │
└────────────────┴───────────────┘


In [24]:
# --- 7. Customer Loyalty (Second Purchase Barrier - 97% one-time buyers) ---
print("\n--- Insight 7: Customer Loyalty (Second Purchase Barrier) ---")
# print("\nPandas Implementation:")
# customer_order_counts_pd = orders_loyalty_df_pd.groupby('customer_unique_id')['order_id'].nunique()
# one_time_buyers_pd = (customer_order_counts_pd == 1).sum()
# repeat_buyers_pd = (customer_order_counts_pd > 1).sum()
# total_customers_for_loyalty_pd = customer_order_counts_pd.count()
# one_time_percentage_pd = (one_time_buyers_pd / total_customers_for_loyalty_pd) * 100
# print(f"Total Unique Customers (for loyalty): {total_customers_for_loyalty_pd}")
# print(f"Customers with 1 order: {one_time_buyers_pd}")
# print(f"Customers with >1 order: {repeat_buyers_pd}")
# print(f"Percentage of customers with only one order: {one_time_percentage_pd:.1f}%")

print("\nPolars Implementation:")
customer_order_counts_pl = orders_loyalty_df_pl.group_by('customer_unique_id').agg(
    pl.n_unique('order_id').alias('order_count')
)
one_time_buyers_pl = customer_order_counts_pl.filter(pl.col('order_count') == 1).shape[0]
repeat_buyers_pl = customer_order_counts_pl.filter(pl.col('order_count') > 1).shape[0]
total_customers_for_loyalty_pl = customer_order_counts_pl.shape[0]
one_time_percentage_pl = (one_time_buyers_pl / total_customers_for_loyalty_pl) * 100
print(f"Total Unique Customers (for loyalty): {total_customers_for_loyalty_pl}")
print(f"Customers with 1 order: {one_time_buyers_pl}")
print(f"Customers with >1 order: {repeat_buyers_pl}")
print(f"Percentage of customers with only one order: {one_time_percentage_pl:.1f}%")


--- Insight 7: Customer Loyalty (Second Purchase Barrier) ---

Polars Implementation:
Total Unique Customers (for loyalty): 96096
Customers with 1 order: 93099
Customers with >1 order: 2997
Percentage of customers with only one order: 96.9%


In [34]:
# --- 8. Delivery Expectation vs. Satisfaction ---
print("\n--- Insight 8: Delivery Expectation vs. Satisfaction ---")
# print("\nPandas Implementation:")
# orders_df_pd['order_estimated_delivery_date'] = orders_df_pd['order_purchase_timestamp'] + pd.to_timedelta(np.random.randint(5, 15, orders_df_pd.shape[0]), unit='D')
# orders_df_pd['order_delivered_customer_date'] = orders_df_pd.apply(
#     lambda row: row['order_estimated_delivery_date'] + pd.to_timedelta(np.random.choice([-2, -1, 0, 1, 2], p=[0.1, 0.2, 0.4, 0.2, 0.1]), size=1, replace=True)[0], axis=1
# )
# delivery_merged_pd = pd.merge(orders_df_pd[['order_id', 'order_estimated_delivery_date', 'order_delivered_customer_date']],
#                             order_reviews_df_pd[['order_id', 'review_score']], on='order_id', how='inner')

# def get_delivery_status_pd(row):
#     if pd.isna(row['order_delivered_customer_date']):
#         return 'Not Delivered'
#     if row['order_delivered_customer_date'] <= row['order_estimated_delivery_date']:
#         return 'On Time/Early'
#     else:
#         return 'Late'

# delivery_merged_pd['delivery_status'] = delivery_merged_pd.apply(get_delivery_status_pd, axis=1)
# avg_rating_by_delivery_pd = delivery_merged_pd.groupby('delivery_status')['review_score'].mean()
# print("Average Rating by Delivery Status (Pandas):\n", avg_rating_by_delivery_pd)

print("\nPolars Implementation:")
orders_df_pl = orders_df_pl.with_columns(
    (pl.col('order_purchase_timestamp') + pl.duration(days=pl.Series(np.random.randint(5, 15, orders_df_pl.shape[0])))).alias('order_estimated_delivery_date')
)
delivery_merged_pl = orders_df_pl.select(['order_id', 'order_estimated_delivery_date', 'order_delivered_customer_date']).join(
    order_reviews_df_pl.select(['order_id', 'review_score']), on='order_id', how='inner'
)

delivery_merged_pl = delivery_merged_pl.with_columns(
    pl.when(pl.col('order_delivered_customer_date').is_null())
    .then(pl.lit('Not Delivered'))
    .when(pl.col('order_delivered_customer_date') <= pl.col('order_estimated_delivery_date'))
    .then(pl.lit('On Time/Early'))
    .otherwise(pl.lit('Late'))
    .alias('delivery_status')
)
avg_rating_by_delivery_pl = delivery_merged_pl.group_by('delivery_status').agg(pl.mean('review_score'))
print("Average Rating by Delivery Status (Polars):\n", avg_rating_by_delivery_pl)


--- Insight 8: Delivery Expectation vs. Satisfaction ---

Polars Implementation:
Average Rating by Delivery Status (Polars):
 shape: (3, 2)
┌─────────────────┬──────────────┐
│ delivery_status ┆ review_score │
│ ---             ┆ ---          │
│ str             ┆ f64          │
╞═════════════════╪══════════════╡
│ Late            ┆ 3.983434     │
│ On Time/Early   ┆ 4.382627     │
│ Not Delivered   ┆ 1.761257     │
└─────────────────┴──────────────┘


In [35]:
# --- 9. Product Mix Optimization (Revenue vs. Volume per Category) ---
print("\n--- Insight 9: Product Mix Optimization ---")
# print("\nPandas Implementation:")
# merged_products_revenue_pd = pd.merge(order_items_df_pd, products_df_pd, on='product_id', how='left')
# category_metrics_pd = merged_products_revenue_pd.groupby('product_category_name').agg(
#     total_revenue=('price', 'sum'),
#     order_count=('order_id', 'nunique')
# ).sort_values(by='total_revenue', ascending=False)
# category_metrics_pd['avg_revenue_per_order'] = category_metrics_pd['total_revenue'] / category_metrics_pd['order_count']
# print("Product Category Metrics (Pandas):\n", category_metrics_pd.head())

print("\nPolars Implementation:")
merged_products_revenue_pl = order_items_df_pl.join(products_df_pl, on='product_id', how='left')
category_metrics_pl = merged_products_revenue_pl.group_by('product_category_name').agg(
    pl.sum('price').alias('total_revenue'),
    pl.n_unique('order_id').alias('order_count')
).with_columns(
    (pl.col('total_revenue') / pl.col('order_count')).alias('avg_revenue_per_order')
).sort('total_revenue', descending=True)
print("Product Category Metrics (Polars):\n", category_metrics_pl.head())


--- Insight 9: Product Mix Optimization ---

Polars Implementation:
Product Category Metrics (Polars):
 shape: (5, 4)
┌────────────────────────┬───────────────┬─────────────┬───────────────────────┐
│ product_category_name  ┆ total_revenue ┆ order_count ┆ avg_revenue_per_order │
│ ---                    ┆ ---           ┆ ---         ┆ ---                   │
│ str                    ┆ f64           ┆ u32         ┆ f64                   │
╞════════════════════════╪═══════════════╪═════════════╪═══════════════════════╡
│ beleza_saude           ┆ 1.2587e6      ┆ 8836        ┆ 142.449224            │
│ relogios_presentes     ┆ 1.2050e6      ┆ 5624        ┆ 214.261323            │
│ cama_mesa_banho        ┆ 1.0370e6      ┆ 9417        ┆ 110.118794            │
│ esporte_lazer          ┆ 988048.97     ┆ 7720        ┆ 127.985618            │
│ informatica_acessorios ┆ 911954.32     ┆ 6689        ┆ 136.336421            │
└────────────────────────┴───────────────┴─────────────┴───────────────

In [36]:
# --- 10. Seller Performance (Impact on Customer Ratings) ---
print("\n--- Insight 10: Seller Performance (Impact on Customer Ratings) ---")
# print("\nPandas Implementation:")
# orders_df_pd['order_approved_at'] = orders_df_pd['order_purchase_timestamp'] + pd.to_timedelta(np.random.randint(0, 3, orders_df_pd.shape[0]), unit='D')
# seller_review_merge_pd = pd.merge(order_items_df_pd[['order_id', 'seller_id']],
#                                   orders_df_pd[['order_id', 'order_purchase_timestamp', 'order_approved_at']],
#                                   on='order_id', how='left')
# seller_review_merge_pd = pd.merge(seller_review_merge_pd,
#                                   order_reviews_df_pd[['order_id', 'review_score']],
#                                   on='order_id', how='left')

# seller_review_merge_pd['dispatch_time_days'] = (seller_review_merge_pd['order_approved_at'] - seller_review_merge_pd['order_purchase_timestamp']).dt.days

# avg_review_by_dispatch_pd = seller_review_merge_pd.groupby('dispatch_time_days')['review_score'].mean().reset_index()
# print("Average Review Score by Dispatch Time (Pandas):\n", avg_review_by_dispatch_pd)

print("\nPolars Implementation:")
orders_df_pl = orders_df_pl.with_columns(
    (pl.col('order_purchase_timestamp') + pl.duration(days=pl.Series(np.random.randint(0, 3, orders_df_pl.shape[0])))).alias('order_approved_at')
)
seller_review_merge_pl = order_items_df_pl.select(['order_id', 'seller_id']).join(
    orders_df_pl.select(['order_id', 'order_purchase_timestamp', 'order_approved_at']),
    on='order_id',
    how='left'
).join(
    order_reviews_df_pl.select(['order_id', 'review_score']),
    on='order_id',
    how='left'
)

seller_review_merge_pl = seller_review_merge_pl.with_columns(
    ((pl.col('order_approved_at') - pl.col('order_purchase_timestamp')).dt.days()).alias('dispatch_time_days')
)

avg_review_by_dispatch_pl = seller_review_merge_pl.group_by('dispatch_time_days').agg(pl.mean('review_score'))
print("Average Review Score by Dispatch Time (Polars):\n", avg_review_by_dispatch_pl)


--- Insight 10: Seller Performance (Impact on Customer Ratings) ---

Polars Implementation:
Average Review Score by Dispatch Time (Polars):
 shape: (3, 2)
┌────────────────────┬──────────────┐
│ dispatch_time_days ┆ review_score │
│ ---                ┆ ---          │
│ i64                ┆ f64          │
╞════════════════════╪══════════════╡
│ 0                  ┆ 4.029775     │
│ 2                  ┆ 4.037146     │
│ 1                  ┆ 4.030513     │
└────────────────────┴──────────────┘
